binoculars take temp.png as input, and takes out from there the last doc, img1, img2 and error

In [3]:
from ipywidgets import widgets
from IPython.display import display as ipython_display
from pyperiscope import Scope
from PIL import Image
import io
import markdown
import pickle
import dill
import pyautogui
import time
import pyperclip

class Binoculars:
    def __init__(self, width=400):
        self.width = width
        self.image1 = None
        self.image2 = None
        self.box = None
        self._create_widgets()

    def load_screenshot_with_data(self, filepath):
        global step
        image = Image.open(filepath)
        
        try:
            # Extract serialized data from metadata
            serialized_data = bytes.fromhex(image.info.get("custom_data", ""))
            data_dict = pickle.loads(serialized_data)
        except (KeyError, pickle.UnpicklingError):
            data_dict = {}  # Return empty dict if no valid data found

        if not data_dict == {}:
            step = Scope(saved_dict=data_dict['step'])
            self.current_step = data_dict['current_step']
            self.last_error = data_dict['last_error']
            self.scope_cell_id = data_dict['scope_cell_id']
            self.current_doc = data_dict['current_doc']
        
    def _create_widgets(self):
        """Initialize all widgets including text areas"""
        # Create description text widget (Markdown)
        self.description = widgets.HTML(
            value='',
            placeholder='Description',
            description='',
        )
        
        # Create image widgets
        self.widget1 = widgets.Image(format='png', width=self.width)
        self.widget2 = widgets.Image(format='png', width=self.width)
        self.image_box = widgets.HBox([self.widget1, self.widget2])
        
        # Create error text widget (Markdown)
        self.error_text = widgets.HTML(
            value='',
            placeholder='Error messages',
            description='',
        )
        
        # Stack all widgets vertically
        self.box = widgets.VBox([
            self.description,
            self.image_box,
            self.error_text
        ])
        
        ipython_display(self.box)
        
    def _convert_to_bytes(self, pil_image):
        """Convert PIL image to bytes"""
        buf = io.BytesIO()
        pil_image.save(buf, format='PNG')
        return buf.getvalue()
    
    def update_image(self, image, position=1):
        """Update either the first or second image"""
        if not isinstance(image, Image.Image):
            raise TypeError("Input must be a PIL Image")
            
        image_bytes = self._convert_to_bytes(image)
        if position == 1:
            self.image1 = image
            self.widget1.value = image_bytes
        elif position == 2:
            self.image2 = image
            self.widget2.value = image_bytes
        else:
            raise ValueError("Position must be 1 or 2")
    
    def update_both(self, image1, image2):
        """Update both images at once"""
        self.update_image(image1, 1)
        self.update_image(image2, 2)
    
    def set_description(self, text):
        """Update description text with markdown formatting"""
        # Convert markdown to HTML
        md = markdown.Markdown()
        self.description.value = md.convert(text)
    
    def set_error(self, text):
        """Update error text with markdown formatting"""
        # Convert markdown to HTML with red color for errors
        md = markdown.Markdown()
        self.error_text.value = md.convert(text)
    
    def clear_texts(self):
        """Clear both description and error texts"""
        self.description.value = ''
        self.error_text.value = ''
    
    def get_current_images(self):
        """Return current PIL images"""
        return self.image1, self.image2

    def update_from_screenshot(self, filepath):
        global step, current_doc, image
        image = Image.open(filepath)
        
        try:
            # Extract serialized data from metadata
            serialized_data = bytes.fromhex(image.info.get("custom_data", ""))
            data_dict = pickle.loads(serialized_data)
        except (KeyError, pickle.UnpicklingError):
            data_dict = {}  # Return empty dict if no valid data found
    
        if not data_dict == {}:
            step = Scope(saved_dict=data_dict['step'])
            step.found_locations = data_dict['last_found']
            #self.current_step = data_dict['current_step']
            last_error = data_dict['last_error']
            #self.scope_cell_id = data_dict['scope_cell_id']
            current_doc = data_dict['current_doc']
            current = Scope(mouse_offset=step.found_locations[0]['mouse_offset'], area_offset=step.area_offset, area_size=step.area_size, saved_image=image)
    
            # update UI
            self.update_both(step.render_preview(), current.render_preview())
            if current_doc:
                self.set_description(current_doc)
            if last_error:
                self.set_error(last_error)

In [4]:
b = Binoculars()

In [5]:
b.update_from_screenshot("_dav_subtitle2.png")

In [2]:
%pip install markdown

Note: you may need to restart the kernel to use updated packages.
